In [16]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

# 데이터셋 가져오기

In [17]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

## 데이터 사이즈 변경

np.expand_dims(data, -1)
- data 위치에 변경하려는 배열
- 차원을 추가할 위치(-1이면 가장 뒷부분)

tf.expand_dims(data, -1)
- np.expand_dims과 동일

data[..., tf.newaxis]
- []안에 기존의 배열을 입력후 원하는 위치에 tf.newaxis로 차원 추가

# 데이터 섞기 및 배치 생성

In [18]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# Keras 모델 서브클래싱 이용 모델 생성

In [30]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)
    
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

# Optimizer 와 손실 함수 선택

In [31]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

# 모델의 loss와 성능을 측정할 지표 선택

In [32]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# 모델 훈련

In [36]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior durint training versus inference (e.g. Dropout)
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

# 모델 테스트

In [37]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [38]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.13173337280750275, Accuracy: 96.05166625976562, Test Loss: 0.0597846619784832, Test Accuracy: 98.07999420166016
Epoch 2, Loss: 0.04148609936237335, Accuracy: 98.72000122070312, Test Loss: 0.04956857115030289, Test Accuracy: 98.37999725341797
Epoch 3, Loss: 0.022392962127923965, Accuracy: 99.26499938964844, Test Loss: 0.05113769322633743, Test Accuracy: 98.47000122070312
Epoch 4, Loss: 0.013676670379936695, Accuracy: 99.56500244140625, Test Loss: 0.057750433683395386, Test Accuracy: 98.41999816894531
Epoch 5, Loss: 0.008958975784480572, Accuracy: 99.69166564941406, Test Loss: 0.05824363976716995, Test Accuracy: 98.32999420166016
